In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

from copy import deepcopy
import itertools
import pickle

import pandas as pd
from scipy.io import savemat, loadmat
import seaborn as sns
from sklearn.metrics import (log_loss, mean_squared_error, roc_curve, auc,
                             precision_recall_fscore_support, confusion_matrix)
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from tqdm import tqdm, trange, tqdm_notebook as tqdmn

from BEKML import BEMKL, plot_distplot
from utils import poly_kernel, gauss_kernel, scoring, plot_kernel_importances, plot_compare_models

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

Populating the interactive namespace from numpy and matplotlib


/home/mtambos/anaconda/envs/pml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('data/wpbc.csv', names=list(range(35)), index_col=0, na_values='?')
display(data.describe())
data = data.dropna()
display(data.head())
data.describe()

,2,3,4,5,6,7,8,9,10,11,...,25,26,27,28,29,30,31,32,33,34
count,198.000000,198.000000,198.00000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,...,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,194.000000
mean,46.732323,17.412323,22.27601,114.856566,970.040909,0.102681,0.142648,0.156243,0.086776,0.192754,...,140.347778,1404.958586,0.143921,0.365102,0.436685,0.178778,0.323404,0.090828,2.847475,3.211340
std,34.462870,3.161676,4.29829,21.383402,352.149215,0.012522,0.049898,0.070572,0.033877,0.027437,...,28.892279,586.006972,0.022004,0.163965,0.173625,0.045181,0.075161,0.021172,1.937964,5.479276
min,1.000000,10.950000,10.38000,71.900000,361.600000,0.074970,0.046050,0.023980,0.020310,0.130800,...,85.100000,508.100000,0.081910,0.051310,0.023980,0.028990,0.156500,0.055040,0.400000,0.000000
25%,14.000000,15.052500,19.41250,98.160000,702.525000,0.093900,0.110200,0.106850,0.063670,0.174075,...,118.075000,947.275000,0.129325,0.248700,0.322150,0.152650,0.275950,0.076578,1.500000,0.000000
50%,39.500000,17.290000,21.75000,113.700000,929.100000,0.101900,0.131750,0.151350,0.086075,0.189350,...,136.500000,1295.000000,0.141850,0.351300,0.402350,0.179250,0.310300,0.086890,2.500000,1.000000
75%,72.750000,19.580000,24.65500,129.650000,1193.500000,0.110975,0.172200,0.200500,0.103925,0.209325,...,159.875000,1694.250000,0.154875,0.423675,0.541050,0.207125,0.358800,0.101375,3.500000,4.000000
max,125.000000,27.220000,39.28000,182.100000,2250.000000,0.144700,0.311400,0.426800,0.201200,0.304000,...,232.200000,3903.000000,0.222600,1.058000,1.170000,0.290300,0.663800,0.207500,10.000000,27.000000


,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,,,,,,,
119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5.0
8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2.0
842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0.0
843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0.0
843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0


,2,3,4,5,6,7,8,9,10,11,...,25,26,27,28,29,30,31,32,33,34
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,...,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,46.938144,17.402320,22.300979,114.781495,969.092268,0.102774,0.142642,0.156309,0.086808,0.192885,...,140.136907,1401.756701,0.143921,0.364567,0.436010,0.178449,0.322251,0.090777,2.867526,3.211340
std,34.523646,3.171672,4.335292,21.430694,353.159959,0.012607,0.050229,0.070942,0.033962,0.027679,...,28.826843,587.040705,0.022092,0.165528,0.174757,0.045529,0.074068,0.021371,1.950588,5.479276
min,1.000000,10.950000,10.380000,71.900000,361.600000,0.074970,0.046050,0.023980,0.020310,0.130800,...,85.100000,508.100000,0.081910,0.051310,0.023980,0.028990,0.156500,0.055040,0.400000,0.000000
25%,14.250000,15.052500,19.342500,98.160000,702.525000,0.093900,0.109850,0.106075,0.063760,0.174075,...,117.925000,940.575000,0.129325,0.247550,0.322150,0.152225,0.275950,0.076368,1.500000,0.000000
50%,39.500000,17.290000,21.795000,113.700000,929.100000,0.102200,0.131750,0.152050,0.086075,0.189350,...,136.500000,1295.000000,0.141750,0.350450,0.401150,0.178500,0.310300,0.086540,2.500000,1.000000
75%,73.000000,19.580000,24.782500,129.650000,1193.500000,0.111375,0.172200,0.200500,0.103925,0.209550,...,159.875000,1694.250000,0.154450,0.423675,0.550175,0.207125,0.358475,0.101775,3.500000,4.000000
max,125.000000,27.220000,39.280000,182.100000,2250.000000,0.144700,0.311400,0.426800,0.201200,0.304000,...,232.200000,3903.000000,0.222600,1.058000,1.170000,0.290300,0.663800,0.207500,10.000000,27.000000


In [3]:
X = data.iloc[:, 1:].copy()
y = data.iloc[:, 0].copy()
N, D = X.shape
y.loc[y == 'N'] = -1
y.loc[y == 'R'] = 1
y = y.astype(int)
display(X.shape, y.shape, y.mean())

(194, 33)

(194,)

-0.5257731958762887

In [4]:
proc_data = data.copy()
proc_data.iloc[:, 0] = y
proc_data = proc_data.loc[:, list(proc_data.columns[1:]) + [proc_data.columns[0]]]
proc_data.to_csv('data/proc_wpbc.csv')
proc_data.head()

,2,3,4,5,6,7,8,9,10,11,...,26,27,28,29,30,31,32,33,34,1
0,,,,,,,,,,,,,,,,,,,,,
119513,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5.0,-1
8423,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2.0,-1
842517,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0.0,-1
843483,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0.0,-1
843584,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0,1


In [5]:
rbf_init, rbf_end = -3, 7
ply_init, ply_end = 1, 4

kernel_attrs = [('rbf', 'all', i) for i in range(rbf_init, rbf_end)]
kernels = [lambda A, B: gauss_kernel(A, B, 2**i)
           for i in range(rbf_init, rbf_end)]

kernel_attrs += [('poly', 'all', i) for i in range(ply_init, ply_end)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i)
            for i in range(ply_init, ply_end)]

kernel_attrs += [('rbf', j, i) for i in range(rbf_init, rbf_end)
                 for j in range(D)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i)
            for i in range(rbf_init, rbf_end) for j in range(D)]

kernel_attrs += [('poly', j, i) for i in range(ply_init, ply_end)
                 for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i)
            for i in range(ply_init, ply_end) for j in range(D)]
len(kernels), len(kernel_attrs)

(442, 442)

In [6]:
X_train, X_test, y_train, y_test =\
        train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [7]:
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.fit_transform(X_test)

# Baseline Model

In [8]:
max_iter = 200
base_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                   hyp_gamma_alpha=1, hyp_gamma_beta=1,
                   hyp_omega_alpha=1, hyp_omega_beta=1,
                   e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                   filter_kernels=False, filter_sv=False, verbose=False,
                   max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
base_model = make_pipeline(Normalizer(), base_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
base_cv_results = cross_validate(base_model, X, y, cv=folds, scoring=scoring)
base_stats = deepcopy(scoring.stats)
base_cv_results, base_stats

0 - Kernels: 102/442 (0.23076923076923078). SV: 135/145 (0.9310344827586207). Mean e: 0.0190. Median e: -0.0025. Std e: 0.0854. 
1 - Kernels: 442/442 (1.0). SV: 125/145 (0.8620689655172413). Mean e: 0.0150. Median e: -0.0219. Std e: 0.0832. 
2 - Kernels: 442/442 (1.0). SV: 131/146 (0.8972602739726028). Mean e: 0.0122. Median e: -0.0301. Std e: 0.0928. 
3 - Kernels: 442/442 (1.0). SV: 138/146 (0.9452054794520548). Mean e: 0.0124. Median e: -0.0273. Std e: 0.0838. 
4 - Kernels: 112/442 (0.25339366515837103). SV: 142/145 (0.9793103448275862). Mean e: 0.0158. Median e: 0.0026. Std e: 0.0498. 
5 - Kernels: 442/442 (1.0). SV: 124/145 (0.8551724137931035). Mean e: 0.0137. Median e: -0.0232. Std e: 0.0834. 
6 - Kernels: 102/442 (0.23076923076923078). SV: 117/146 (0.8013698630136986). Mean e: 0.0140. Median e: -0.0079. Std e: 0.0742. 
7 - Kernels: 442/442 (1.0). SV: 128/146 (0.8767123287671232). Mean e: 0.0162. Median e: -0.0212. Std e: 0.0823. 
8 - Kernels: 102/442 (0.23076923076923078). SV: 1

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([ 7.8249,  7.2214,  6.9914,  6.7585,  9.0194,  9.433 ,  9.6853,  6.301 ,  6.3321,  6.8171,
          6.6031,  6.9476,  6.5834,  6.4954, 10.2711,  9.4091,  9.3368,  7.5572,  7.4025,  9.9642]),
  'score_time': array([0.4824, 0.482 , 0.4795, 0.5464, 0.481 , 0.5553, 0.5617, 0.4716, 0.4716, 0.4681, 0.4862,
         0.4777, 0.4812, 0.4848, 0.5049, 0.5136, 0.5201, 0.4924, 0.4845, 0.4936]),
  'test_score': array([0.7755, 0.7143, 0.7917, 0.7708, 0.7755, 0.7551, 0.8125, 0.7292, 0.6939, 0.7755, 0.7917,
         0.7917, 0.7551, 0.7347, 0.8125, 0.7917, 0.7959, 0.7347, 0.7708, 0.7708]),
  'train_score': array([0.8   , 0.8   , 0.7808, 0.7671, 0.7586, 0.7931, 0.7671, 0.8014, 0.7931, 0.7724, 0.7877,
         0.7945, 0.7931, 0.7793, 0.774 , 0.774 , 0.8   , 0.7862, 0.7808, 0.7808])},
 [{'elapsed_time': 7.529919624328613,
   'nr_kernels_used': 102,
   'nr_sv_used': 135,
   'total_kernels': 442,
   'total_sv': 145},
  {'elapsed_time': 6.929396390914917,
   'nr_kernels_used': 442,
   'nr

In [9]:
base_times = np.array([s['elapsed_time'] for s in base_stats])
base_kernels = np.array([s['nr_kernels_used'] for s in base_stats])
base_sv = np.array([s['nr_sv_used'] for s in base_stats])
print(
    f"Score: {base_cv_results['test_score'].mean()} +- {base_cv_results['test_score'].std()}\n"
    f"Time: {base_times.mean()} +- {base_times.std()}\n"
    f"Kernels: {base_kernels.mean()} +- {base_kernels.std()}\n"
    f"SVs: {base_sv.mean()} +- {base_sv.std()}"
)

Score: 0.7671768707482993 +- 0.031304010662793466
Time: 7.544500803947448 +- 1.3451217364245294
Kernels: 336.05 +- 151.47490716286973
SVs: 126.2 +- 10.052860289489752


# Kernel-sparse model

In [10]:
max_iter = 200
ksparse_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1e-11, hyp_omega_beta=1e9,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ksparse_pipeline = make_pipeline(Normalizer(), ksparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ksparse_cv_results = cross_validate(ksparse_pipeline, X, y, cv=folds, scoring=scoring)
ksparse_stats = deepcopy(scoring.stats)
ksparse_cv_results, ksparse_stats

0 - Kernels: 1/442 (0.0022624434389140274). SV: 135/145 (0.9310344827586207). Mean e: 0.0057. Median e: 0.0005. Std e: 0.1097. 
1 - Kernels: 1/442 (0.0022624434389140274). SV: 117/145 (0.8068965517241379). Mean e: 0.0068. Median e: 0.0006. Std e: 0.1184. 
2 - Kernels: 3/442 (0.006787330316742082). SV: 142/146 (0.9726027397260274). Mean e: 0.0084. Median e: 0.0005. Std e: 0.1646. 
3 - Kernels: 2/442 (0.004524886877828055). SV: 131/146 (0.8972602739726028). Mean e: 0.0070. Median e: 0.0006. Std e: 0.5107. 
4 - Kernels: 3/442 (0.006787330316742082). SV: 144/145 (0.993103448275862). Mean e: 0.0068. Median e: 0.0004. Std e: 0.1354. 
5 - Kernels: 1/442 (0.0022624434389140274). SV: 117/145 (0.8068965517241379). Mean e: 0.0062. Median e: 0.0006. Std e: 0.1056. 
6 - Kernels: 3/442 (0.006787330316742082). SV: 137/146 (0.9383561643835616). Mean e: 0.0068. Median e: 0.0005. Std e: 0.1317. 
7 - Kernels: 2/442 (0.004524886877828055). SV: 142/146 (0.9726027397260274). Mean e: 0.0088. Median e: 0.0004

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([ 9.5997, 11.9649,  7.367 ,  6.7854,  6.5825,  6.4706,  6.4704,  6.5929,  6.7238,  7.9169,
          7.4974,  7.2699,  6.4387,  6.6478,  6.6132,  6.8652,  6.6199,  6.8778,  6.7172,  6.5813]),
  'score_time': array([0.6096, 0.6605, 0.4722, 0.4719, 0.4754, 0.4894, 0.4654, 0.4731, 0.4868, 0.4674, 0.4695,
         0.4747, 0.4834, 0.4723, 0.4929, 0.4702, 0.4676, 0.4789, 0.4762, 0.4811]),
  'test_score': array([0.7551, 0.7755, 0.7708, 0.7917, 0.7551, 0.7551, 0.7917, 0.8125, 0.7755, 0.7551, 0.75  ,
         0.7708, 0.7347, 0.7755, 0.8125, 0.7708, 0.7551, 0.7755, 0.6875, 0.8125]),
  'train_score': array([0.7724, 0.7931, 0.8014, 0.7808, 0.7862, 0.8   , 0.7671, 0.774 , 0.7655, 0.7862, 0.7877,
         0.8014, 0.8   , 0.7931, 0.7808, 0.7534, 0.7931, 0.7862, 0.8082, 0.7603])},
 [{'elapsed_time': 9.303120374679565,
   'nr_kernels_used': 1,
   'nr_sv_used': 135,
   'total_kernels': 442,
   'total_sv': 145},
  {'elapsed_time': 11.661324262619019,
   'nr_kernels_used': 1,
   'nr_sv

In [11]:
ksparse_times = np.array([s['elapsed_time'] for s in ksparse_stats])
ksparse_kernels = np.array([s['nr_kernels_used'] for s in ksparse_stats])
ksparse_sv = np.array([s['nr_sv_used'] for s in ksparse_stats])
print(
    f"Score: {ksparse_cv_results['test_score'].mean()} +- {ksparse_cv_results['test_score'].std()}\n"
    f"Time: {ksparse_times.mean()} +- {ksparse_times.std()}\n"
    f"Kernels: {ksparse_kernels.mean()} +- {ksparse_kernels.std()}\n"
    f"SVs: {ksparse_sv.mean()} +- {ksparse_sv.std()}"
)

Score: 0.7691539115646259 +- 0.028310221234628365
Time: 6.931991732120514 +- 1.296048386007398
Kernels: 1.85 +- 0.9096702699330126
SVs: 131.8 +- 9.657121724406295


# SV-sparse model

In [12]:
max_iter = 200
ssparse_model = BEMKL(kernels=kernels,
                      hyp_lambda_alpha=1e-11, hyp_lambda_beta=1e9,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1, hyp_omega_beta=1,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ssparse_pipeline = make_pipeline(Normalizer(), ssparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ssparse_cv_results = cross_validate(ssparse_pipeline, X, y, cv=folds, scoring=scoring)
ssparse_stats = deepcopy(scoring.stats)
ssparse_cv_results, ssparse_stats

0 - Kernels: 432/442 (0.9773755656108597). SV: 64/145 (0.4413793103448276). Mean e: 0.0155. Median e: -0.0127. Std e: 0.0686. 
1 - Kernels: 432/442 (0.9773755656108597). SV: 117/145 (0.8068965517241379). Mean e: 0.0134. Median e: -0.0157. Std e: 0.0655. 
2 - Kernels: 432/442 (0.9773755656108597). SV: 78/146 (0.5342465753424658). Mean e: 0.0137. Median e: -0.0107. Std e: 0.0627. 
3 - Kernels: 442/442 (1.0). SV: 77/146 (0.5273972602739726). Mean e: 0.0159. Median e: -0.0188. Std e: 0.0721. 
4 - Kernels: 442/442 (1.0). SV: 98/145 (0.6758620689655173). Mean e: 0.0162. Median e: -0.0299. Std e: 0.0979. 
5 - Kernels: 442/442 (1.0). SV: 129/145 (0.8896551724137931). Mean e: 0.0140. Median e: -0.0172. Std e: 0.0704. 
6 - Kernels: 102/442 (0.23076923076923078). SV: 54/146 (0.3698630136986301). Mean e: 0.0155. Median e: -0.0067. Std e: 0.0627. 
7 - Kernels: 432/442 (0.9773755656108597). SV: 122/146 (0.8356164383561644). Mean e: 0.0115. Median e: -0.0115. Std e: 0.0604. 
8 - Kernels: 102/442 (0.2

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([8.8349, 7.1368, 6.9665, 7.1308, 6.7762, 7.0169, 6.7713, 7.0331, 6.6652, 6.6842, 6.6136,
         6.7939, 6.9516, 7.0909, 6.65  , 6.8005, 6.7512, 6.8086, 6.8925, 6.7739]),
  'score_time': array([0.4959, 0.4842, 0.4695, 0.4783, 0.488 , 0.4844, 0.4836, 0.4814, 0.4796, 0.4638, 0.4829,
         0.4731, 0.4743, 0.4774, 0.4814, 0.4883, 0.4834, 0.484 , 0.487 , 0.4782]),
  'test_score': array([0.7551, 0.7959, 0.7917, 0.7708, 0.7755, 0.7755, 0.7708, 0.7917, 0.7347, 0.7347, 0.7917,
         0.7708, 0.7347, 0.7755, 0.7917, 0.7708, 0.7551, 0.7755, 0.7708, 0.75  ]),
  'train_score': array([0.8   , 0.7724, 0.7808, 0.7808, 0.8   , 0.7793, 0.7945, 0.7534, 0.8   , 0.7931, 0.7671,
         0.774 , 0.8069, 0.7724, 0.774 , 0.8082, 0.7862, 0.7793, 0.7808, 0.7877])},
 [{'elapsed_time': 8.549424648284912,
   'nr_kernels_used': 432,
   'nr_sv_used': 64,
   'total_kernels': 442,
   'total_sv': 145},
  {'elapsed_time': 6.844874143600464,
   'nr_kernels_used': 432,
   'nr_sv_used': 117,
   't

In [13]:
ssparse_times = np.array([s['elapsed_time'] for s in ssparse_stats])
ssparse_kernels = np.array([s['nr_kernels_used'] for s in ssparse_stats])
ssparse_sv = np.array([s['nr_sv_used'] for s in ssparse_stats])
print(
    f"Score: {ssparse_cv_results['test_score'].mean()} +- {ssparse_cv_results['test_score'].std()}\n"
    f"Time: {ssparse_times.mean()} +- {ssparse_times.std()}\n"
    f"Kernels: {ssparse_kernels.mean()} +- {ssparse_kernels.std()}\n"
    f"SVs: {ssparse_sv.mean()} +- {ssparse_sv.std()}"
)

Score: 0.769153911564626 +- 0.01903103921496427
Time: 6.658322393894196 +- 0.4620291422876375
Kernels: 314.55 +- 157.2944611230796
SVs: 88.35 +- 34.10465510747763


In [14]:
import json
with open('wpbc_results.json', 'w') as fp:
    json.dump(
        {
            'ksparse': {
                'scores': list(ksparse_cv_results['test_score']),
                'times': list(ksparse_times),
                'kernels': [int(k) for k in ksparse_kernels],
                'svs': [int(s) for s in ksparse_sv],
            },
            'ssparse': {
                'scores': list(ssparse_cv_results['test_score']),
                'times': list(ssparse_times),
                'kernels': [int(k) for k in ssparse_kernels],
                'svs': [int(s) for s in ssparse_sv],
            },
            'base': {
                'scores': list(base_cv_results['test_score']),
                'times': list(base_times),
                'kernels': [int(k) for k in base_kernels],
                'svs': [int(s) for s in base_sv],
            },
            'total_kernels': len(kernels),
            'total_sv': len(X_train),
        },
        fp,
        indent=4,
        sort_keys=True
    )